## Выбираем лучшие настройки для тикеров

In [1]:
%reload_ext autoreload
%autoreload 2

In [ ]:

from app.cache import LocalCache
from bot.vis import TaskProgress
from bot import TestAlgorithm
from app.config import RunConfig
from concurrent.futures import ThreadPoolExecutor, as_completed

test_configs = [
    (RunConfig(
        ticker=t_config.ticker,
        step_max_cnt=max_shares,
        step_base_cnt=base_shares,
        step_lots=1,

        majority_trade=t_config.majority_trade,
        pretest_period=pretest_period,
        pretest_type=RunConfig.PRETEST_PRE,  # todo

        threshold_buy_steps=0,
        threshold_sell_steps=0,
        stop_up_p=stop_up_p,
        stop_down_p=0,

        step_size=t_config.step_size + step_size_shift,
        step_set_orders_cnt=step_cnt,
    ))
    for t_config in [
        # RunConfig.from_repr_string('DELI+ 3/pre6:0/2 x l2 x 1.6¤ |u0.01 d0.0|'),
        # RunConfig.from_repr_string('ETLN- 3/pre4:0/2 x l10 x 1.0¤'),
        # RunConfig.from_repr_string('EUTR+ 3/pre3:-3/3 x l2 x 0.8¤'),
        # RunConfig.from_repr_string('RNFT+ 3/pre5:3/2 x l2 x 1.4¤'),
        RunConfig.from_repr_string('SPBE- 3/pre5:0/2 x l2 x 1.8¤'),


        # RunConfig.from_repr_string('GEMC- 3/pre6:0/2 x l10 x 2.4¤'),
        # RunConfig.from_repr_string('HNFG- 3/rsi13:0/2 x l10 x 3.2¤'),
        # RunConfig.from_repr_string('UWGN- 3/pre6:0/2 x l10 x 1.2¤'),
        # RunConfig.from_repr_string('GLTR- 3/pre6:0/2 x l10 x 5.4¤'),
        # RunConfig.from_repr_string('POLY- 3/pre6:0/2 x l10 x 3.4¤'),
        # RunConfig.from_repr_string('ASTR- 3/pre6:0/2 x l10 x 1.4¤'),
        # RunConfig.from_repr_string('QIWI- 3/pre6:0/2 x l10 x 2.6¤'),
    ]
    # for max_shares in [3]
    # for base_shares in [0]
    # for stop_up_p in [0]
    # for step_size_shift in [0] # [0, .2, -.2, -.4, .4]
    # for step_cnt in [2]  # вот тут некоторым удобнее в 2. можно потестить в вариаторе 2 и 3
    # for pretest_period in [5]  # range(3, 7) # todo


    for max_shares in [3, 4]
    for base_shares in [0]
    for stop_up_p in [0, 0.01]
    for step_size_shift in [0, .2, -.2]
    for step_cnt in [2]
    for pretest_period in range(3, 7)
]

# test_configs = [
#     RunConfig.from_repr_string('SPBE- 3/pre5:1/2 x l1 x 1.0¤'),
#     RunConfig.from_repr_string('SPBE- 3/pre5:0/2 x l1 x 1.0¤'),
# ]

def run_test(config: RunConfig):
    test_alg = TestAlgorithm(do_printing=False, config=config, use_cache=True)
    return test_alg.test(
        last_test_date=None,
        test_days_num=20,  # todo
        shares_count=0,

        auto_conf_days_freq=1,
        auto_conf_prev_days=config.pretest_period,
    )

unique_configs = set(test_configs)
# unique_configs = test_configs

results = []
LocalCache.clear()
progress = TaskProgress(len(unique_configs))


# todo на первом проходе надо 1 ставить, тогда запросы к API будут только 1 раз
with ThreadPoolExecutor(max_workers=4) as executor:
    future_to_params = {executor.submit(run_test, config): config for config in unique_configs}
    
    for future in as_completed(future_to_params):
        params = future_to_params[future]
        res = future.result()
        if res:
            results.append(res)
        progress.update_progress()


print(f"cache {LocalCache.get_counter('cache_miss')} vals, used {LocalCache.get_counter('cache_find')} times")

# Вывод результатов или их дальнейшая обработка
sorted_results = sorted(results, key=lambda x: (x['config'].ticker, -float(x['profit_p'])), reverse=False)

print()
for item in sorted_results:
    print(item)

Запуск в 02:04
[████--------------------------] 12% (6/48), закончим через "20m 43s" в 02:27          

тестируем 48 вариантов на 14 дней с полным кэшированием и только рабочими днями

трекаем время и качество - топ и низ показателей

```
Закончено в 01:38, количество 48, длительность 0:10:48
cache 29727 vals, used 93195 times

{'profit': 46.1, 'profit_p': 12.89, 'pot_p': 51.22, 'config': SPBE- 3/pre4:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 137}
{'profit': 58.5, 'profit_p': 12.36, 'pot_p': 12.19, 'config': SPBE- 4/pre4:0/2 x l1 x 1.8¤ , 'op': 174}
{'profit': 54.4, 'profit_p': 11.72, 'pot_p': 3.89, 'config': SPBE- 4/pre4:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 181}
...
{'profit': -8.9, 'profit_p': -1.79, 'pot_p': 0, 'config': SPBE- 4/pre3:0/2 x l1 x 1.6¤ , 'op': 80}
{'profit': -9.5, 'profit_p': -1.91, 'pot_p': 0, 'config': SPBE- 4/pre3:0/2 x l1 x 2.0¤ |u0.01 d0.0| , 'op': 28}
{'profit': -9.4, 'profit_p': -2.52, 'pot_p': 0, 'config': SPBE- 3/pre3:0/2 x l1 x 2.0¤ , 'op': 31}
```

потом аналогично, но для всех дней

```
Запуск в 01:41
Закончено в 01:47, количество 48, длительность 0:06:04
cache 17050 vals, used 42463 times

{'profit': 22.5, 'profit_p': 6.49, 'pot_p': 1.92, 'config': SPBE- 3/pre5:0/2 x l1 x 1.8¤ , 'op': 30}
{'profit': 20.8, 'profit_p': 5.73, 'pot_p': 0, 'config': SPBE- 3/pre6:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 49}
{'profit': 19.2, 'profit_p': 5.42, 'pot_p': 1.21, 'config': SPBE- 3/pre4:0/2 x l1 x 1.8¤ |u0.01 d0.0| , 'op': 31}
...
{'profit': -9.4, 'profit_p': -1.98, 'pot_p': -2.61, 'config': SPBE- 4/pre5:0/2 x l1 x 2.0¤ , 'op': 23}
{'profit': -9.0, 'profit_p': -2.53, 'pot_p': -3.33, 'config': SPBE- 3/pre4:0/2 x l1 x 2.0¤ , 'op': 16}
{'profit': -16.6, 'profit_p': -3.43, 'pot_p': 0, 'config': SPBE- 4/pre3:0/2 x l1 x 1.8¤ , 'op': 27}
```

смотрим на время

и смотрим на результаты - если получаем плюс-минус одинаковые результаты, то можно брать быстрый алгоритм. если нет, то точно планируем полный обход

И правильнее сравнивать 10 рабочих дней и 14 нерабочих тогда уже

```
Запуск в 01:51
Закончено в 01:59, количество 48, длительность 0:07:59
cache 22690 vals, used 64304 times

{'profit': 26.0, 'profit_p': 7.35, 'pot_p': 5.42, 'config': SPBE- 3/pre6:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 124}
{'profit': 34.6, 'profit_p': 7.34, 'pot_p': 5.41, 'config': SPBE- 4/pre5:0/2 x l1 x 2.0¤ |u0.01 d0.0| , 'op': 48}
{'profit': 32.0, 'profit_p': 6.75, 'pot_p': 8.89, 'config': SPBE- 4/pre6:0/2 x l1 x 2.0¤ |u0.01 d0.0| , 'op': 25}
...
{'profit': -10.1, 'profit_p': -2.13, 'pot_p': -0.44, 'config': SPBE- 4/pre3:0/2 x l1 x 2.0¤ |u0.01 d0.0| , 'op': 17}
{'profit': -8.6, 'profit_p': -2.43, 'pot_p': -1.79, 'config': SPBE- 3/pre3:0/2 x l1 x 2.0¤ , 'op': 20}
{'profit': -14.6, 'profit_p': -3.1, 'pot_p': -2.28, 'config': SPBE- 4/pre3:0/2 x l1 x 1.6¤ , 'op': 55}
```

---

вывод - всё очень разное и с ходу не выделяется никаких закономерностей
будем пробовать

только рабочие дни 30

Запуск в 00:55
Закончено в 00:58, количество 2, длительность 0:02:57
cache 2448 vals, used 3068 times

{'profit': 33.0, 'profit_p': 9.8, 'pot_p': 1.36, 'config': SPBE- 3/pre5:0/2 x l1 x 1.0¤ , 'op': 212}
{'profit': 32.8, 'profit_p': 9.74, 'pot_p': 1.35, 'config': SPBE- 3/pre5:1/2 x l1 x 1.0¤ , 'op': 209}

"все" 30
Запуск в 01:00
Закончено в 01:01, количество 2, длительность 0:01:09
cache 1084 vals, used 946 times

{'profit': 18.3, 'profit_p': 5.47, 'pot_p': 0.69, 'config': SPBE- 3/pre5:1/2 x l1 x 1.0¤ , 'op': 129}
{'profit': 18.3, 'profit_p': 5.47, 'pot_p': 0.69, 'config': SPBE- 3/pre5:0/2 x l1 x 1.0¤ , 'op': 129}

рабочие 14

Запуск в 01:06
Закончено в 01:07, количество 2, длительность 0:01:28
cache 1130 vals, used 1479 times

{'profit': 5.4, 'profit_p': 1.46, 'pot_p': 0, 'config': SPBE- 3/pre5:0/2 x l1 x 1.0¤ , 'op': 114}
{'profit': 3.1, 'profit_p': 0.84, 'pot_p': 0, 'config': SPBE- 3/pre5:1/2 x l1 x 1.0¤ , 'op': 114}

все 14

Запуск в 01:05
Закончено в 01:06, количество 2, длительность 0:00:49
cache 723 vals, used 472 times

{'profit': 3.4, 'profit_p': 0.98, 'pot_p': 0.26, 'config': SPBE- 3/pre5:1/2 x l1 x 1.0¤ , 'op': 88}
{'profit': 3.4, 'profit_p': 0.98, 'pot_p': 0.26, 'config': SPBE- 3/pre5:0/2 x l1 x 1.0¤ , 'op': 88}

промежуточный вариант
{'profit': 18.0, 'profit_p': 5.17, 'pot_p': 1.4, 'config': SPBE- 3/pre5:0/2 x l1 x 1.8¤ , 'op': 35}

--

На едином конфиге
без кэша: сек, длительность 0:00:43
{'profit': 18.0, 'profit_p': 5.17, 'pot_p': 1.4, 'config': SPBE- 3/pre5:0/2 x l1 x 1.8¤ , 'op': 35}

с кэшом: 22 сек, cache 865 vals, used 1073 times
{'profit': 16.6, 'profit_p': 4.77, 'pot_p': 1.29, 'config': SPBE- 3/pre5:0/2 x l1 x 1.8¤ , 'op': 33}

без кэшей - длительность 0:01:58
{'profit': 11.9, 'profit_p': 3.34, 'pot_p': 4.41, 'config': SPBE- 3/pre5:0/2 x l1 x 1.8¤ , 'op': 32}

только кэш свечей для тикера - длительность 0:00:41
{'profit': 11.9, 'profit_p': 3.34, 'pot_p': 4.41, 'config': SPBE- 3/pre5:0/2 x l1 x 1.8¤ , 'op': 32}

все кэши включены -



Для 1 запуска - SPBE- 3/pre5:0/2 x l2 x 1.8¤ - длительность 0:01:59
{'profit': 18.0, 'profit_p': 5.17, 'pot_p': 1.4, 'config': SPBE- 3/pre5:0/2 x l1 x 1.8¤ , 'op': 35}

он же контрольный при переписывании

Для полноценного запуска на 14 дней - 43 мин
Запуск в 17:59
Закончено в 18:45, количество 48, длительность 0:45:46

{'profit': 18.0, 'profit_p': 5.17, 'pot_p': 1.4, 'config': SPBE- 3/pre5:0/2 x l1 x 1.6¤ , 'op': 35}
{'profit': 18.0, 'profit_p': 5.17, 'pot_p': 1.4, 'config': SPBE- 3/pre5:0/2 x l1 x 1.8¤ , 'op': 35}
{'profit': 23.3, 'profit_p': 5.02, 'pot_p': 1.35, 'config': SPBE- 4/pre4:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 45}
{'profit': 22.7, 'profit_p': 4.89, 'pot_p': 1.32, 'config': SPBE- 4/pre4:0/2 x l1 x 1.8¤ |u0.01 d0.0| , 'op': 46}
{'profit': 15.6, 'profit_p': 4.48, 'pot_p': 1.21, 'config': SPBE- 3/pre4:0/2 x l1 x 1.6¤ , 'op': 32}
{'profit': 20.8, 'profit_p': 4.48, 'pot_p': 1.21, 'config': SPBE- 4/pre5:0/2 x l1 x 1.6¤ , 'op': 39}
{'profit': 20.8, 'profit_p': 4.48, 'pot_p': 1.21, 'config': SPBE- 4/pre4:0/2 x l1 x 1.6¤ , 'op': 40}
{'profit': 20.8, 'profit_p': 4.48, 'pot_p': 1.21, 'config': SPBE- 4/pre5:0/2 x l1 x 1.8¤ , 'op': 39}
{'profit': 19.1, 'profit_p': 4.12, 'pot_p': 1.11, 'config': SPBE- 4/pre3:0/2 x l1 x 1.8¤ |u0.01 d0.0| , 'op': 35}
{'profit': 19.0, 'profit_p': 4.09, 'pot_p': 1.1, 'config': SPBE- 4/pre5:0/2 x l1 x 1.8¤ |u0.01 d0.0| , 'op': 46}
{'profit': 13.9, 'profit_p': 3.99, 'pot_p': 1.08, 'config': SPBE- 3/pre6:0/2 x l1 x 1.8¤ , 'op': 27}
{'profit': 13.9, 'profit_p': 3.99, 'pot_p': 1.08, 'config': SPBE- 3/pre6:0/2 x l1 x 1.6¤ , 'op': 27}
{'profit': 13.2, 'profit_p': 3.79, 'pot_p': 1.02, 'config': SPBE- 3/pre5:0/2 x l1 x 2.0¤ , 'op': 27}
{'profit': 16.7, 'profit_p': 3.6, 'pot_p': 0.97, 'config': SPBE- 4/pre6:0/2 x l1 x 1.8¤ , 'op': 31}
{'profit': 16.7, 'profit_p': 3.6, 'pot_p': 0.97, 'config': SPBE- 4/pre6:0/2 x l1 x 1.6¤ , 'op': 31}
{'profit': 16.0, 'profit_p': 3.45, 'pot_p': 0.93, 'config': SPBE- 4/pre6:0/2 x l1 x 2.0¤ |u0.01 d0.0| , 'op': 26}
{'profit': 11.5, 'profit_p': 3.3, 'pot_p': 0.89, 'config': SPBE- 3/pre6:0/2 x l1 x 2.0¤ |u0.01 d0.0| , 'op': 27}
{'profit': 11.2, 'profit_p': 3.22, 'pot_p': 0.87, 'config': SPBE- 3/pre6:0/2 x l1 x 2.0¤ , 'op': 26}
{'profit': 11.2, 'profit_p': 3.22, 'pot_p': 0.87, 'config': SPBE- 3/pre4:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 46}
{'profit': 14.4, 'profit_p': 3.1, 'pot_p': 0.84, 'config': SPBE- 4/pre4:0/2 x l1 x 2.0¤ , 'op': 28}
{'profit': 14.2, 'profit_p': 3.06, 'pot_p': 0.83, 'config': SPBE- 4/pre3:0/2 x l1 x 1.6¤ , 'op': 38}
{'profit': 10.5, 'profit_p': 3.02, 'pot_p': 0.81, 'config': SPBE- 3/pre4:0/2 x l1 x 1.8¤ , 'op': 30}
{'profit': 14.0, 'profit_p': 3.02, 'pot_p': 0.81, 'config': SPBE- 4/pre5:0/2 x l1 x 2.0¤ , 'op': 27}
{'profit': 13.8, 'profit_p': 2.97, 'pot_p': 0.8, 'config': SPBE- 4/pre5:0/2 x l1 x 2.0¤ |u0.01 d0.0| , 'op': 32}
{'profit': 13.4, 'profit_p': 2.89, 'pot_p': 0.78, 'config': SPBE- 4/pre6:0/2 x l1 x 1.8¤ |u0.01 d0.0| , 'op': 31}
{'profit': 10.0, 'profit_p': 2.87, 'pot_p': 0.78, 'config': SPBE- 3/pre6:0/2 x l1 x 1.8¤ |u0.01 d0.0| , 'op': 31}
{'profit': 9.9, 'profit_p': 2.84, 'pot_p': 0.77, 'config': SPBE- 3/pre4:0/2 x l1 x 2.0¤ , 'op': 28}
{'profit': 12.7, 'profit_p': 2.74, 'pot_p': 0.74, 'config': SPBE- 4/pre6:0/2 x l1 x 2.0¤ , 'op': 27}
{'profit': 12.7, 'profit_p': 2.74, 'pot_p': 0.74, 'config': SPBE- 4/pre5:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 41}
{'profit': 12.0, 'profit_p': 2.59, 'pot_p': 0.7, 'config': SPBE- 4/pre3:0/2 x l1 x 1.8¤ , 'op': 37}
{'profit': 11.4, 'profit_p': 2.46, 'pot_p': 0.66, 'config': SPBE- 4/pre4:0/2 x l1 x 1.8¤ , 'op': 32}
{'profit': 9.5, 'profit_p': 2.05, 'pot_p': 0.55, 'config': SPBE- 4/pre4:0/2 x l1 x 2.0¤ |u0.01 d0.0| , 'op': 32}
{'profit': 7.1, 'profit_p': 2.04, 'pot_p': 0.55, 'config': SPBE- 3/pre3:0/2 x l1 x 2.0¤ |u0.01 d0.0| , 'op': 28}
{'profit': 8.9, 'profit_p': 1.92, 'pot_p': 0.52, 'config': SPBE- 4/pre3:0/2 x l1 x 2.0¤ , 'op': 30}
{'profit': 6.6, 'profit_p': 1.9, 'pot_p': 0.51, 'config': SPBE- 3/pre3:0/2 x l1 x 2.0¤ , 'op': 27}
{'profit': 4.2, 'profit_p': 1.21, 'pot_p': 0.33, 'config': SPBE- 3/pre3:0/2 x l1 x 1.8¤ , 'op': 30}
{'profit': 3.8, 'profit_p': 1.09, 'pot_p': 0.29, 'config': SPBE- 3/pre5:0/2 x l1 x 2.0¤ |u0.01 d0.0| , 'op': 22}
{'profit': 3.4, 'profit_p': 0.98, 'pot_p': 0.26, 'config': SPBE- 3/pre3:0/2 x l1 x 1.8¤ |u0.01 d0.0| , 'op': 30}
{'profit': 3.3, 'profit_p': 0.95, 'pot_p': 0.26, 'config': SPBE- 3/pre6:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 57}
{'profit': 3.1, 'profit_p': 0.89, 'pot_p': 0.24, 'config': SPBE- 3/pre5:0/2 x l1 x 1.8¤ |u0.01 d0.0| , 'op': 26}
{'profit': 1.6, 'profit_p': 0.46, 'pot_p': 0.12, 'config': SPBE- 3/pre4:0/2 x l1 x 1.8¤ |u0.01 d0.0| , 'op': 27}
{'profit': 1.5, 'profit_p': 0.43, 'pot_p': 0.12, 'config': SPBE- 3/pre3:0/2 x l1 x 1.6¤ , 'op': 32}
{'profit': 2.0, 'profit_p': 0.43, 'pot_p': 0.12, 'config': SPBE- 4/pre3:0/2 x l1 x 2.0¤ |u0.01 d0.0| , 'op': 27}
{'profit': 1.3, 'profit_p': 0.37, 'pot_p': 0.1, 'config': SPBE- 3/pre5:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 41}
{'profit': 1.2, 'profit_p': 0.34, 'pot_p': 0.09, 'config': SPBE- 3/pre4:0/2 x l1 x 2.0¤ |u0.01 d0.0| , 'op': 28}
{'profit': 0.7, 'profit_p': 0.2, 'pot_p': 0.05, 'config': SPBE- 3/pre3:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 41}
{'profit': 0.6, 'profit_p': 0.13, 'pot_p': 0.03, 'config': SPBE- 4/pre6:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 61}
{'profit': -1.6, 'profit_p': -0.34, 'pot_p': -0.09, 'config': SPBE- 4/pre3:0/2 x l1 x 1.6¤ |u0.01 d0.0| , 'op': 43}

In [ ]:
a = 2

#82 Динамическая подстройка параметров запуска
---

исходный вариант алгоритма
{'profit': 549.7, 'profit_p': 55.41, 'config': RNFT 4/pre13:0/3 x l2 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+ , 'op_cnt': 2318}

на pre
{'profit': 253.8, 'profit_p': 51.17, 'config': RNFT 4/pre13:0/3 x l1 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+ , 'op_cnt': 2318}

last_test_date='2024-05-01',
test_days_num=365,  # 365,
auto_conf_days_freq=7,
auto_conf_prev_days=7,
{'profit': 42.5, 'profit_p': 8.57, 'config': RNFT 4/0/3 x l1 x 1.2¤, |s0 b0| |u0.0 d0.0| maj+z+ , 'op_cnt': 2283}



